In [8]:
# Dependencies
import requests
import json
import gmaps
import gmaps.geojson_geometries
import gmaps.datasets
import pandas as pd
import numpy as np
from matplotlib.cm import viridis
from matplotlib.colors import to_hex
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [9]:
# MN COVID-19 Data (needs to be updated weekly)
# csv downloaded from https://www.health.state.mn.us/diseases/coronavirus/stats/wcrmap.csv
covid_data = pd.read_csv("wcrmap.csv")

our_counties_df = covid_data[(covid_data['County'].isin(['Anoka','Washington','Ramsey']))]
lastweek = our_counties_df.iloc[:, [5]]
lastweek.columns = ['']
twoweeks = our_counties_df.iloc[:, [4]]
twoweeks.columns = ['']
case_rate = lastweek.add(twoweeks, fill_value=0)
case_rate
counties = our_counties_df['County'].to_frame()
case_rate_df = counties.join(case_rate, how='outer')
case_rate_df

,County,
1,Anoka,15.5
61,Ramsey,16.5
81,Washington,14.9


In [10]:
# Find schools
school_names = ["Karner Blue Education Center",
                "Quora Education Center",
                "Bellaire Education Center",
                "Cambia Hills East Bethel",
                "Pankalo Education Center",
                "916 Mahtomedi Academy",
                "Metro Heights Academy",
                "WELS North",
                "WBL South Campus",
                "Aris Clinic Currell",
                "Aris Clinic Woodwinds",
                "Career and Technical Center"]

locations = []

# Loop through schools
for school in school_names:
    base_url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={school}&inputtype=textquery&fields=formatted_address,name,geometry&key={g_key}"
    response = requests.get(base_url)
    data = response.json()
    lat = data['candidates'][0]['geometry']['location']['lat']
    lng = data['candidates'][0]['geometry']['location']['lng']
    coords = (lat, lng)
    locations.append(coords)

In [12]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
#counties_geojson = gmaps.geojson_geometries.load_geometry('us-counties')
with open('mn_counties.json') as f:
    geometry = json.load(f)

fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
gini_layer = gmaps.geojson_layer(
    geometry,
    fill_color=['red','green','blue']
)
fig.add_layer(gini_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…